# Adding Durability with Temporal

You've just built a research application that generates PDF reports. It works perfectly—until it doesn't.

Imagine this: Your application conducts expensive research through an LLM call (costing time and money), but then **crashes** during PDF generation due to a network ourFW. When you restart, everything is lost. You're back to the beginning, paying for the same LLM call again, making your users wait, and burning through your API budget.

As these workflows grow more complex—chaining multiple LLM calls, database queries, external APIs—the problem compounds. Every failure means starting over completely.

In this section, we'll solve this problem by making your application durable. You'll learn how to build Gen-AI applications that survive failures, recover automatically, and never lose progress.

## Setup Your Notebook

Run the following code blocks to install various packages and tools necessary to run this notebook.


In [24]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio litellm reportlab python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /Users/azhou/Desktop/edu-ai-workshop-mcp/env/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Create a `.env` File

Next you'll create a `.env` file to store your API keys.
In the file browser on the left, create a new file and name it `.env`.
Note that this file doesn't persist across notebooks or sesions.

**Note**: It may disappear as soon as you create it. This is because Google Collab hides hidden files (files that start with a `.`) by default.
To make this file appear, click the icon that is a crossed out eye and hidden files will appear.

Then double click on the `.env` file and add the following line with your API key.

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = "openai/gpt-4o"
```

By default this notebook uses OpenAI's GPT-4o.
If you want to use a different LLM provider, look up the appropriate model name [in their documentation](https://docs.litellm.ai/docs/providers) and change the `LLM_MODEL` field and provide your API key.

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

# Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings

import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

In [25]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [26]:
# Running this will download the Temporal CLI, which we need for this workshop.

!curl -sSf https://temporal.download/cli.sh | sh

temporal: Downloading Temporal CLI latest
temporal: Temporal CLI installed at /Users/azhou/.temporalio/bin/temporal
temporal: For convenience, we recommend adding it to your PATH
temporal: If using bash, run echo export PATH="\$PATH:/Users/azhou/.temporalio/bin" >> ~/.bashrc


In [ ]:
# Mermaid renderer, run at the beginning to setup rendering of diagrams
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    """
    Renders a Mermaid diagram in Google Colab using mermaid.ink.

    Args:
        graph_definition (str): The Mermaid diagram code (e.g., "graph LR; A-->B;").
    """
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

# Adding Durability

## What Can Go Wrong with Gen-AI Applications?

Let's brainstorm the issues you might face when running your research application from Notebook 1 in production.

**Think about these categories:**
* **Technical failures:** What external services could fail?
* **Timing issues:** What if something takes longer than expected?
* **Recovery challenges:** If something breaks halfway through, what happens?

*Take 2 minutes to discuss with your neighbor, then we'll share answers.*

## Common Issues with Gen-AI Applications in Production

**Common answers we typically hear:**
* LLM API timeouts or rate limiting
* PDF generation fails due to disk space
* Network connectivity issues
* Process crashes mid-execution
* Restarting burns money

## These Aren't New Problems

The challenges you just identified? They're the same problems we've been solving in distributed systems for decades:

**Your Research Application in Production Reality:**
* **LLM API call** - External service that can timeout, rate limit, or be down.
* **PDF generation** - File system operation that can fail due to disk space
* **User input/output** - Network operations that can be interrupted



## Gen-AI Applications are Distributed Systems!

**This is a distributed system!** Your "simple" application is actually:
* Multiple network calls to external services
* File system operations
* State that needs to persist across failures
* Coordination between different steps

**The challenge:** Traditional distributed systems tools weren't designed for AI workflows. They don't understand expensive LLM calls, context windows, or long-term state management.

**The good news:** You can use a platform that guarantees the _reliable execution_ of your code.

## Your Report Generation Application Needs Durable Execution

Recall your research application from Notebook 1? Here's what happens in production:

**Scenario:** User asks for research on "sustainable energy trends"
1. LLM call succeeds - generates comprehensive research content ($2.50 in API costs)
2. PDF generation fails - disk full, permission error, or process crash
3. **User has to start over completely** - losing expensive work and time

## What Developers Actually Want

* "Just fix the disk issue and generate the PDF from the research you already have."
* "Don't make me pay for the same LLM call twice!"
* "Don't lose my work because of a simple file system error!"

## What Normal Execution Gives Us

* Every failure means restarting from scratch
* Expensive LLM calls are repeated unnecessarily
* User experience becomes frustrating and unreliable
* No way to resume from where you left off

## What We Need

A way to make our AI applications resilient to these failures.

## What Is Durable Execution?

* Crash-proof execution
* Retries upon failure
* Maintains application state, resuming after a crash at the point of failure
* Can run across a multitude of processes, even on different machines
  * Virtualizes execution

## Temporal Provides Durable Execution

* It removes the pain of plumbing your distributed system by handling state, retries, timeouts, state preservation right out of the box
* Open-Source MIT Licensed
* Code based approach to Workflow design
  - Instead of building custom orchestration systems, you write normal functions.
  - Since it’s a general purpose programming language, there are no abstractions to get in your way. Since AI patterns will continue to evolve, general-purpose programming languages will be as well-suited to implement these new patterns.
* Use your own tools, processes, and libraries
* Support for 7 languages
  * Python, TypeScript, Ruby, Java, Go, PHP, .NET

## Demo (Expand for instructor notes or to run on your own)
<!--
Normal Execution Demo:
1. To demonstrate the power of durable execution, we'll first show the power of running the app with no durable execution. This is the code that we showed in the first notebook.
2. Clone this repository: `https://github.com/temporalio/edu-ai-workshop-agentic-loop`. The instructions will also be in the README.
2. From the `src/module_one_01_ai_agentic_loop/app.py` directory, run `app.py` with `python app.py`.
3. When prompted, provide the prompt you want to prompt OpenAI in the command line.
4. Before the process generates a PDF, kill the process.
5. Rerun the application again with `python app.py` and show that the process restarted and you have to have your application start the research again. Emphasize that from a cost perspective, this could be very costly, because you could have to re-run through many tokens to get to where you left off.

Durable Execution Demo:
1. Now show the durable version by switching into the ``src/module_one_02_adding_durability` directory.
2. Run the Worker with `python worker.py`.
3. Run the Workflow with `python workflow.py`.
4. When prompted, provide the prompt you want to prompt OpenAI in the command line.
5. Before the process generates a PDF, kill the Worker.
6. Rerun the Worker and show that you continue right where you left off.
7. Emphasize that you lost no progress or data. The Workflow will continue by generating the PDF (available in the same directory) and completing the process successfully.
10. Show the Workflow Execution completion in the Web UI.
-->

## Durable Execution Requirements

Temporal relies on a Replay mechanism to recover from failure.
As your program progresses, Temporal saves the input and output from function calls to the history.
This allows a failed program to restart right where it left off.
This can also save us a lot of money since we aren't re-burning through tokens!

For example:

User request: "Research sustainable energy trends"
✓ Step 1: LLM research call → Output saved to history
✓ Step 2: Generate summary → Output saved to history  
✗ Step 3: Create PDF → CRASH!

On restart:
- Temporal replays Steps 1 & 2 from history (no actual execution)
- Continues from Step 3 with the same inputs

**Because of this, Temporal requires your workflow to be deterministic**

A Workflow is deterministic if it produces the same output given the same input.

## _Wait, how can AI code be deterministic?_

Your **workflow** needs to be deterministic, not the entire application.

The key is understanding Temporal's separation of concerns.

1. **Non-deterministic parts** - Run arbitrary code that has the potential to fail due to external conditions
  * Ex: Calling LLMs, accessing the file system, writing to a database.
  * Take 1 minute to discuss with your neighbor any other examples, then we'll share answers
2. **Deterministic parts** - Orchestrate the non-deterministic parts
  * Ex: Branching, looping, mathematical operations, etc.

## Consider the Following Example

* Depending on the time of day, a different decision is made
* If it's 5:00pm, it's dinner time
* If it's 9:30am, it's breakfast time

**What would happen if a user ran this application at 11:59am, it crashed and was replayed at 12:01pm? What would the user expect?**

In [ ]:
diagram = """
graph TD
    A["Get Current Time"] --> B["Is am or pm?"]
    B --> C["Time for breakfast"]
    B --> D["Time for dinner"]
"""
render_mermaid(diagram)


## _What Does This Have to Do with AI?_

_Common Misconception_: "Since workflows need to be deterministic, your AI applications will always behave the same way and follow identical paths."

_Reality_: **This statement is completely wrong.**

### **Determistic != predetermined**
Deterministic means your workflow makes the same decisions when replayed with the same inputs and external responses. Your AI application can still be dynamic and adaptive!

## AI Application Reality Check

**Common Fear:** "If my workflow is deterministic, my AI application will always do the same thing."

**Reality:** Your application can be completely dynamic while still being deterministic.

Example: User asks "Research best Italian restaurant in New York City."
LLM returns product information → application follows restaurant research path

**The deterministic guarantee**: If any of these workflows need to replay because of a network outage or an application crash, they will execute the exact same steps with the exact same LLM responses recorded in their history. But each scenario can produce completely different, contextually dynamic results.

**Bottom line**: Determinism ensures reliability, not rigidity. Your AI remains as smart and adaptive as you design it to be.

## AI Research application Examples

**Each run is completely different** (dynamic), but **each individual run is reproducible** (deterministic).

In [ ]:
diagram = """
graph TD
    A["Ask AI for a Plan"] --> B{"Make Tea"}
    B --> C["Boil Water"]
    C --> D["Steep tea"]
    D --> E["Remove and enjoy"]
"""
render_mermaid(diagram)


In [ ]:
diagram = """
graph TD
    A["Ask AI for a Plan"] --> B{"Slay a dragon"}
    B --> C["Find the weak spot"]
    C --> D["Acquire the correct weapon"]
    D --> E["Carry out your attack"]
"""
render_mermaid(diagram)

In [ ]:
diagram = """
graph TD
    A["Ask AI for a Plan"] --> B{"Write Code"}
    B --> C["Locate files"]
    C --> D["Write code"]
    D --> E["Evaluate result"]
"""
render_mermaid(diagram)

## Deterministic Workflows Are *Essential* for AI Workflows

* The applicationic Loop: AI applications follow a repeatable pattern of reasoning and action:
  * Evaluate goal - What am I trying to accomplish?
  * Locate tools - What capabilities do I need to use?
  * Execute tools - Perform the actual work (API calls, file operations, etc.)
  * Evaluate completion - Did I achieve the goal or need to continue?
* Tools that the LLM decides to call become **dynamic**, not **non-deterministic**.
  - Dynamic: The LLM can choose different tools based on the situation (flight search vs hotel search vs weather API)
  - Non-Deterministic: The same input produces same outputs on replay
* **Deterministic, not predetermined**

## Let's Make Your application Durable

We're about to transform your simple research application into a durable one. Here's what changes:

* Your tools will become crash-proof
* Automatic retries and recovery
* State persistence

This results in a process such as:
LLM Decision → Tool A → Result X (Saved in history, then on replay, same result X will result in the same next decision) → Next Decision

## What stays the same

* Your core logic (LLM call → PDF generation)
* Your inputs and outputs
* Your business requirements

## Package Our Inputs & Outputs for Ease of Management

For ease of use, evolution of parameters, and type checking, Temporal recommends passing and returning a single object from functions. `dataclass` is the recommended structure here, but anything serializable will work.

In [ ]:
# TODO: Run this code block to load it into the program
from dataclasses import dataclass

@dataclass
class LLMCallInput:
  prompt: str
  llm_api_key: str
  llm_model: str

@dataclass
class PDFGenerationInput:
  content: str
  filename: str = "research_pdf.pdf"

## What is an Activity?

* An Activity is a function/method that is prone to failure and/or non-deterministic.
* Temporal requires all non-deterministic code be run in an Activity

Examples:
  - External API calls - LLM requests, web scraping, database queries
  - File system operations - Reading documents, writing reports, managing storage
  - Network operations - HTTP requests, email sending, data transfers
  - Resource-intensive computations - Image processing, data analysis, model inference

## What Activities Give You

* **Automatic retries** when external code fails
* **Timeout handling** for slow operations and detecting failures
* **Detailed visibility** of execution, including inputs/outputs for debugging
* **Automatic checkpoints** - if your workflow crashes, Activities aren't re-executed. Instead, your Workflow continues from the last known good state



## Tasks/Tools become Activities

To turn a function/method into an Activity, add the `@activity.defn` decorator.

In [ ]:
# TODO: Run this code block to load it into the program
from temporalio import activity
from litellm import completion, ModelResponse

@activity.defn
def llm_call(input: LLMCallInput) -> ModelResponse:
    response = completion(
      model=input.llm_model,
      api_key=input.llm_api_key,
      messages=[{ "content": input.prompt,"role": "user"}]
    )
    return response

In [ ]:
# Step 1: Make the code an Activity. Look at the cell below for the solution.
# Step 2: Now run the code to load it into the program

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

def create_pdf_activity(input: PDFGenerationInput) -> str:
    print("Creating PDF document...")

    doc = SimpleDocTemplate(input.filename, pagesize=letter)
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))
    paragraphs = input.content.split('\n\n')
    for para in paragraphs:
        if para.strip():
          p = Paragraph(para.strip(), styles['Normal'])
          story.append(p)
          story.append(Spacer(1, 12))

    doc.build(story)

    print(f"SUCCESS! PDF created: {input.filename}")
    return input.filename

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_02_Adding_Durability_with_Temporal" / "create_pdf_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Your Code

**Your LLM call is now:**
* Protected against API timeouts
* Automatically retried with backoff
* Observable for debugging

**Your PDF generation is now:**
* Protected against file system errors
* Automatically retried if temporary failures
* Tracked for completion verification

## Activities Are Called from Workflows

- You orchestrate the execution of your Activities from within a Workflow.
- Workflows contain the decision-making flow, but Activities perform the actual work.
- Each Activity call is recorded in the workflow history with inputs and outputs
- Workflows can wait for activity completion, handle failures, and make decisions based on results

In [ ]:
diagram = """
graph TD
    A["Research application Workflow"] --> B["Activity: LLM Call"]
    B --> C["Workflow: Process LLM Response"]
    C --> D["Activity: Generate Image"]
    D --> E["Workflow: Combine Content & Image"]
    E --> F["Activity: PDF Creation"]
    F --> G["Workflow Complete"]

    style A fill:#e1f5fe
    style B fill:#fff3e0
    style D fill:#fff3e0
    style F fill:#fff3e0
    style C fill:#e8f5e8
    style E fill:#e8f5e8
"""
render_mermaid(diagram)

## More Input/Output Packaging

Just like with Activities, Temporal recommends passing a single object to the Workflow for input and returning a single object.

In [ ]:
# TODO: Run this code block to load it into the program
from dataclasses import dataclass

@dataclass
class GenerateReportInput:
    prompt: str

@dataclass
class GenerateReportOutput:
    result: str

## Creating the Workflow

* Activities are orchestrated within a Temporal Workflow.
* Workflows must **not** make API calls, file system calls, or anything non-deterministic. That is what Activities are for.
* Workflows are async, and you define them as a class decorated with the `@workflow.defn` decorator.
* Every Workflow has a **single** entry point, which is an `async` method decorated with `@workflow.run`.

In [ ]:
# Step 1: Notice how the Workflow calls the `llm_call` Activity. 
# Follow the pattern to call the `create_pdf_activity`.
# Step 2: A Start-to-Close timeout is the maximum amount of time a single Activity Execution can take. We recommend always setting this timeout.
# Set a Start-to-Close timeout of 10 seconds for the `create_pdf_activity`.
# Step 3: Run this code block to load it into the program
from datetime import timedelta
from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:

        llm_call_input = LLMCallInput(prompt=input.prompt, llm_api_key=LLM_API_KEY, llm_model=LLM_MODEL)

        research_facts = await workflow.execute_activity(
            llm_call,
            llm_call_input,
            start_to_close_timeout=timedelta(seconds=30), # maximum amount of time a single Activity Execution can take.
        )

        workflow.logger.info("Research complete!")

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename = await workflow.execute_activity(
            # TODO: Call the create_pdf_activity here
            pdf_generation_input,
            # TODO: Set the Start-to-Close timeout of 10 seconds here
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_02_Adding_Durability_with_Temporal" / "generatereportworkflow_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Running a Worker

* Temporal Workflows are run on Workers
* Workers wait for tasks to do, such as an Activity or Workflow Task, and execute them.
* Workers have Workflows and Activities registered to them so the Worker knows what to execute.
* Workers find tasks by listenting on a Task Queue
* This makes the execution of work indirect; _any_ Worker can pick up a registered Workflow or Activity

In [ ]:
# Run this code block to load it into the program
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", namespace="default")

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="research", # the task queue the Worker is polling
            workflows=[GenerateReportWorkflow], # register the Workflow
            activities=[llm_call, create_pdf_activity], # register the Activities
            activity_executor=activity_executor
        )

        print(f"Starting the worker....")
        await worker.run()

## Running a Temporal Service

* The Temporal Service brings it all together
* The Temporal Service can be run locally, self-hosted, or you can use Temporal Cloud
* The service acts as the supervisor of your Workflows, Activities, and everything else

**To run the Temporal Server in this exercise environment**:
1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

## Starting the Worker

A Workflow can't execute if a Worker isn't running.

In [ ]:
# Due to the limitation of Jupyter Notebooks and Google Collab, this is how
# you must start the worker in a Notebook environment
import asyncio

worker = asyncio.create_task(run_worker())

# If you are running this code in a typical Python environment, you can start
# the Worker by just calling `asyncio.run`
# if __name__ == "__main__":
#    asyncio.run(run_worker())

In [ ]:
# Step 1: Set the Task Queue to be the Task Queue that your Worker is polling
# Step 2: Run this code block to load it into the program
from temporalio.client import Client
import uuid

# Create client connected to server at the given address
client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow,
    GenerateReportInput(prompt=prompt),
    id=f"generate-research-report-workflow-{uuid.uuid4()}", # user-defined Workflow identifier, which typically has some business meaning
    task_queue="", # TODO: Set the Task Queue to be the Task Queue that your Worker is polling
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_02_Adding_Durability_with_Temporal" / "worker_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Getting the Result

The example above uses async execution. You can `await` the handle to get the result.

In [ ]:
# Get the result
result = await handle.result()
print(f"Result: {result}")

# To download the report: right click `research_pdf.pdf` in your file explore, then click `Download`.

## Temporal Web UI

- Temporal provides a robust Web UI for managing Workflow Executions
- Can gain insights like responses from Activities, execution time, and failures
- Great for debugging and understanding what's happening during your Workflow Executions.

## Exploring the Web UI

Can you locate the following items on the Web UI?

- The name of the Task Queue
- The name of the two Activities called
- The inputs and outputs of the called Activities
- Input and output of the Workflow Execution

## Simulating Failure

What happens if the Worker process were to crash during execution?

## Adding a Durable Timer

- Timers introduce delays in your Workflow with guaranteed execution.
- Durable timers will fire even if there is no Worker running, and persists despite restarts and infrastructure failures
- Let's add one to the Workflow to give us time to kill the Worker in the middle of execution.

In [ ]:
from datetime import timedelta

from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:

        llm_call_input = LLMCallInput(prompt=input.prompt, llm_api_key=LLM_API_KEY, llm_model=LLM_MODEL)

        research_facts = await workflow.execute_activity(
            llm_call,
            llm_call_input,
            start_to_close_timeout=timedelta(seconds=30),
        )

        workflow.logger.info("Research complete!")

        # Adding a Timer here to pause the Workflow Execution
        await workflow.sleep(timedelta(seconds=20))

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename = await workflow.execute_activity(
            create_pdf_activity,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=10),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

## Restart the Worker

- After a Workflow change, you must restart the Worker for the change to take effect.

In [ ]:
# Run this to kill the current Worker
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

In [ ]:
# Starting the Worker again
import asyncio

worker = asyncio.create_task(run_worker())

# Check if the task is in the set of all tasks
if worker in asyncio.all_tasks():
    # The sleep is necessary because of the async task scheduling in Jupyter
    print("Task is currently active.") # The Worker now registers the updated Workflow changes
else:
    print("Task is not found in active tasks (might have finished or not yet scheduled).")

## Start the Workflow and Simulate an Error

Start the Workflow again, prompt the LLM, wait about ~5 seconds to let the Timer start, then kill the Worker.

In [ ]:
import uuid

client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow.run,
    GenerateReportInput(prompt=prompt),
    id=f"generate-research-report-workflow-{uuid.uuid4()}",
    task_queue="research",
)

print(f"Started workflow. Wait about ~4 seconds to let the Timer start, then kill the Worker.")

In [ ]:
# After about 5 seconds, run this to kill the current Worker
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

## Watch the Progress in the Web UI

Refresh your Web UI, click on the running Workflow Execution, and watch the progress. What do you observe? Does the Timer complete despite the Worker being kiled?

In [ ]:
# Get the Temporal Web UI URL
# The Temporal server is running on port 8080 (configured in devcontainer.json)
print("Temporal Web UI is available on port 8080")
print("Click on the 'Ports' tab and open port 8080, or navigate to http://localhost:8080")

## Restart the Worker to Resume Execution

- Restart the Worker and return to the WebUI.
  * What do you think will happen?
- You will see the Workflow pick up where it left off as if nothing happened!

In [ ]:
# Starting the Worker again
import asyncio

worker = asyncio.create_task(run_worker())

---
# Exercise 2 - Adding Durability

* In these exercises you will:
  * Transform your LLM calls and your execution of tools to Activities
  * Use a Temporal Workflow to orchestrate your Activities
  * Observe how Temporal handles your errors
  * Debug your error and observe your Workflow Execution successfully complete
* Go to the **Exercise** Directory in the Google Drive and open the **Practice** Directory
* Open _02-Adding-Durability.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 5 mins**